In [1]:
import cv2
import numpy as np
import os

In [2]:
# params
exam = '25080225_08_2019_08_37_59'

path_out = '{}/pupil_deep/eye_test/out'.format(os.getcwd().replace('/auxiliaries',''))
full_exam = False

In [3]:
paths = []
if exam == '':
    path = '{}/pupil_deep/eye_test/out/exams'.format(os.getcwd().replace('/auxiliaries',''))
    files = os.listdir(path)
    paths = ['{}/{}'.format(path, x) for x in files]
else:
    paths.append('{}/pupil_deep/eye_test/out/{}'.format(os.getcwd().replace('/auxiliaries',''), exam))

In [4]:
paths

['/media/marcos/Dados/Projects/PupilDeep/pupil_deep/eye_test/out/25080225_08_2019_08_37_59']

In [5]:
number_files = []
os.listdir(paths[0])

for file in os.listdir(paths[0]):
    if 'original_' in file:
        number_files.append(file)

print('quantity: {}, examples: {}'.format(len(number_files), number_files[0: 10: 1]))

quantity: 300, examples: ['original_40.png', 'original_41.png', 'original_42.png', 'original_43.png', 'original_44.png', 'original_45.png', 'original_46.png', 'original_47.png', 'original_48.png', 'original_49.png']


In [6]:
def create_exam(name, list_frames):
    out = cv2.VideoWriter('{}/{}.mp4'.format(path_out, name), cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

    for i in range(len(list_frames)):
        out.write(img_array[i])
    
    out.release()

In [7]:
img_array = []
size = None

for ex in paths:
    print(ex)
    for i in range(1, len(number_files)):
        path_final = 'final_{}.png'.format(i)
        path_binary = 'binary_{}.png'.format(i)
        path_process = 'process_{}.png'.format(i)
        path_original = 'original_{}.png'.format(i)
        path_histogram = 'histogram_{}.png'.format(i)
        path_img_process = 'img_process_{}.png'.format(i)

        im_final = cv2.imread('{}/{}'.format(ex, path_final))
        im_binary = cv2.imread('{}/{}'.format(ex, path_binary))
        im_process = cv2.imread('{}/{}'.format(ex, path_process))
        im_original = cv2.imread('{}/{}'.format(ex, path_original))
        im_histogram = cv2.imread('{}/{}'.format(ex, path_histogram))
        im_img_process = cv2.imread('{}/{}'.format(ex, path_img_process))

        im_pres = cv2.hconcat([im_original, im_process, im_img_process])

        img_array.append(im_pres)

        if size is None:
            height, width, _ = im_pres.shape
            size = (width,height)
        
    if not full_exam:
        tmp = ex.split('/')
        create_exam(tmp[len(tmp)-1], img_array)
        img_array.clear()

if full_exam:
    create_exam('full_exam', img_array)

/media/marcos/Dados/Projects/PupilDeep/pupil_deep/eye_test/out/25080225_08_2019_08_37_59
